In [2]:
from tkinter import *
from PIL import Image as PIL_Image
from PIL import ImageTk
import tkinter.filedialog as tkFileDialog
import cv2
from tkinter import messagebox
import matplotlib.pyplot as plt

In [3]:
# Retrieve the stored variables
try:
    %store -r result_list
    %store -r path
except Exception as e:
    print(f"Error retrieving stored variables: {e}")

# Ensure variables are retrieved correctly
if 'result_list' in globals() and 'path' in globals():
    print("Variables loaded successfully!")
else:
    print("Error: result_list or path not found.")

# Call the function to display the output
try:
    display_output()
except Exception as e:
    print(f"Error displaying output: {e}")


no stored variable or alias result_list
no stored variable or alias path
Error: result_list or path not found.
Error displaying output: name 'master_frame' is not defined


In [4]:
def display_output():
    sub_frame1 = Frame(master_frame, bg='black', pady=5)
    sub_frame2 = Frame(master_frame, bg='brown', pady=5, padx=5)
    sub_frame3 = Frame(master_frame, pady=5, padx=5)
    sub_frame4 = Frame(master_frame, pady=5, padx=5)

    sub_frame1.grid(row=1, column=1, padx=5, pady=5)
    sub_frame2.grid(row=2, column=1, padx=5, pady=5)
    sub_frame3.grid(row=3, column=1, padx=5, pady=5)
    sub_frame4.grid(row=4, column=1, padx=5, pady=5)

    title = Label(master=sub_frame1, text="NEPALI FAKE CURRENCY DETECTOR", fg='dark blue', font="Verdana 28 bold")
    title.pack()

    canvas_input = Canvas(master=sub_frame2, width=675, height=300)
    canvas_input.pack()

    # Modify this part of display_output()
    if len(path) > 0 and path[-4:] == '.jpg':
        image = cv2.imread(path)
        original_image = image.copy()

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (675, 300))
        image = PIL_Image.fromarray(image)

        # Keep a reference to the image object to prevent it from being garbage collected
        canvas_input.image = ImageTk.PhotoImage(image)
        canvas_input.create_image(0, 0, anchor=NW, image=canvas_input.image)

    pass_count = 0

    for i in range(4):
        for j in range(3):
            feature_num = 3 * i + j

            if feature_num < len(result_list):  # Ensure we don't access out of range
                sub_frame4.grid_rowconfigure(i, weight=1)
                sub_frame4.grid_columnconfigure(j, weight=1)

                feature_frame = Frame(master=sub_frame4, relief=RAISED, borderwidth=1, bg='light blue')
                feature_frame.grid(row=i, column=j, padx=20, pady=20, sticky="nsew")

                frame1 = Frame(feature_frame, padx=3, pady=3)
                frame2 = Frame(feature_frame, bg='brown', pady=5, padx=5)
                frame3 = Frame(feature_frame)
                frame4 = Frame(feature_frame)
                frame5 = Frame(feature_frame)

                frame1.grid(row=1, column=1, padx=5, pady=5, ipadx=100)
                frame2.grid(row=2, column=1, padx=5, pady=5)
                frame3.grid(row=3, column=1, padx=5, pady=5)
                frame4.grid(row=4, column=1, padx=5, pady=5)
                frame5.grid(row=5, column=1, padx=5, pady=5)

                label1 = Label(master=frame1, text=f"Feature {feature_num + 1}", fg='black', font="Verdana 12 bold")
                label1.pack()

                canvas = Canvas(master=frame2, width=200, height=200)
                canvas.pack()

                image = result_list[feature_num][0].copy()

                h, w = image.shape[:2]
                aspect_ratio = w / h

                resize_height, resize_width, img_x, img_y = 200, 200, 0, 0
                if h > w:
                    resize_height = 200
                    resize_width = int(aspect_ratio * resize_height)
                    img_x = (200 - resize_width) // 2
                elif h < w:
                    resize_width = 200
                    resize_height = int(resize_width / aspect_ratio)
                    img_y = (200 - resize_height) // 2

                image = cv2.resize(image, (resize_width, resize_height))
                image = PIL_Image.fromarray(image)
                image = ImageTk.PhotoImage(image)

                canvas.image = image  # Keep reference to prevent garbage collection
                canvas.create_image(img_x, img_y, anchor=NW, image=image)
                
                if feature_num < 7:
                    avg_score = "{:.3f}".format(result_list[feature_num][1])
                    text2 = f"Avg. SSIM Score: {avg_score}"
                    label2 = Label(master=frame3, text=text2, fg='dark blue', font="Verdana 11", bg='light blue')
                    label2.pack()
                else:
                    status = result_list[feature_num][1]
                    if status == True:
                        text2 = "6 characters detected!"
                    else:
                        text2 = "Less than 6 characters detected!"
                    label2 = Label(master = frame3, text = text2, fg = 'dark blue', font = "Verdana 11", bg='light blue')
                    label2.pack()
                
                if len(result_list[feature_num]) > 2:
                    max_score = "{:.3f}".format(result_list[feature_num][2])
                    text3 = f"Max. SSIM Score: {max_score}"
                else:
                    text3 = ""

                label3 = Label(master=frame4, text=text3, fg='dark blue', font="Verdana 11", bg='light blue')
                label3.pack()

                status = result_list[feature_num][-1]  # Ensure we access the correct status index
                if status == True:
                    pass_count += 1
                    label4 = Label(master=frame5, text="Status: PASS!", fg='green', font="Verdana 11 bold", bg='light blue')
                else:
                    label4 = Label(master=frame5, text="Status: FAIL!", fg='red', font="Verdana 11 bold", bg='light blue')
                label4.pack()

    result = Label(master=sub_frame3, text=f"RESULT: {pass_count} / 8 features PASSED!", fg='green', font="Verdana 24 bold")
    result.pack()

    # Add the final output message
    if pass_count == 8:
        final_output = "Real Note."
        final_fg_color = 'green'
    else:
        final_output = "Fake Note."
        final_fg_color = 'red'

    final_result = Label(master=sub_frame3, text=final_output, fg=final_fg_color, font="Verdana 24 bold")
    final_result.pack()


    # Configuring the scroll bar widget and canvas widget
def scrollbar_function(event):
    canvas.configure(scrollregion=canvas.bbox("all"),width=1050,height=550)

    
# Main Function
# Declaring root window and specifying its attributes
root=Tk()
root.title('Fake Currency Detection - Result Analysis')

# Defining attributes of root window
root.resizable(False, False)  # This code helps to disable windows from resizing

window_height = 600
window_width = 1100

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

x_cordinate = int((screen_width/2) - (window_width/2))
y_cordinate = int((screen_height/2) - (window_height/2))

root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

# Creating a main frame inside the root window
main_frame=Frame(root,relief=GROOVE, bd=1)
main_frame.place(x=10,y=10) # Placing the frame at (10, 10)

# Creating a canvas inside main_frame
canvas=Canvas(main_frame)
master_frame=Frame(canvas)  # Creating master_frame inside the canvas

# Inserting  and configuringscrollbar widget
myscrollbar=Scrollbar(main_frame,orient="vertical",command=canvas.yview)
canvas.configure(yscrollcommand=myscrollbar.set)

myscrollbar.pack(side="right",fill="y")
canvas.pack(side="left")
canvas.create_window((0,0),window=master_frame,anchor='nw')
master_frame.bind("<Configure>",scrollbar_function)

# Displaying output
display_output()

# Open the root window and loop
root.mainloop()

NameError: name 'path' is not defined